# ELBO implementation and comparisons

I have implemented ELBO for M&M model based on write up in [this document](https://www.overleaf.com/project/5bd111aaa3ec8118d7b1cfa8). See Section 8 for derivation details; also Section B for an independent re-derivation from Yuxin Zou in checking my work.

(the analytic form are not identical due to different simplifications but I coded both up and checked they agree).

## `susieR` implementation in univariate case

Here I simulate one trait and run with `susieR::susie` as well as `mmbr::susie` to check if the elbo agree.

In [1]:
library(mmbr)
set.seed(2)

Loading required package: mashr
Loading required package: ashr


I pick `L=5` and simulate a univariate trait,

In [2]:
L = 5
dat = mmbr_sim1(r=1)

Then run SuSiE and get the ELBO,

In [3]:
res = susieR::susie(dat$X,dat$y,L=L,scaled_prior_variance=0.2,estimate_residual_variance=F,estimate_prior_variance=F)

In [4]:
res$elbo

[1] -429.4050 -409.5493 -407.7077 -407.7058 -407.7058

In [5]:
res = susieR::susie(dat$X,dat$y,L=L,scaled_prior_variance=0.2,estimate_residual_variance=T,estimate_prior_variance=F)
res$elbo

[1] -405.9513 -305.5374 -301.4516 -301.4352 -301.4352

In [6]:
res = susieR::susie(dat$X,dat$y,L=L,scaled_prior_variance=0.2,estimate_residual_variance=T,estimate_prior_variance=T)
res$elbo

[1] -404.3175 -355.2881 -299.9210 -299.3357 -299.3351

## Compare `mmbr`'s Bayesian multivariate regression module with SuSiE

I have implemented the multiple regression [in this class in `mmbr` package](https://github.com/gaow/mmbr/blob/master/R/bayesian_multiple_regression.R). On the `mmbr` interface this is triggered by setting prior `V` as a scalar and make input `Y` a one column matrix,

In [7]:
res = mmbr::msusie(dat$X,dat$y,L=L,prior_variance=0.2,compute_objective=T,estimate_residual_variance=F,estimate_prior_variance=F)
res$elbo

[1] -429.4050 -409.5493 -407.7077 -407.7058 -407.7058

In [8]:
res = mmbr::msusie(dat$X,dat$y,L=L,prior_variance=0.2,compute_objective=T,estimate_residual_variance=T,estimate_prior_variance=F)
res$elbo

[1] -405.9513 -305.5374 -301.4516 -301.4352 -301.4352

In [9]:
res = mmbr::msusie(dat$X,dat$y,L=L,prior_variance=0.2,compute_objective=T,estimate_residual_variance=T,estimate_prior_variance=T)
res$elbo

[1] -404.3175 -355.2881 -299.9210 -299.3357 -299.3351

The output is identical to using `susieR`.

## Compare `mmbr`'s MASH regression module with SuSiE

Here I create a degenerated [MASH regression module implemented in `mmbr`](https://github.com/gaow/mmbr/blob/master/R/mash_multiple_regression.R). "Degenerated" means it has only one phenotype, and the prior is also a trivial one-component 1 by 1 matrix of `0.2 * var(Y)`. Here we use `EE` model in MASH (`alpha = 0`).

The difference in code between this and previous section in computing ELBO can be found in this function for computing the L-th KL: [compute_expected_loglik_partial()](https://github.com/gaow/mmbr/blob/7eead885b914efcb44e97acfb26a1b81bec6ab64/R/single_effect_regression.R#L37) and this function: [compute_objective()](https://github.com/gaow/mmbr/blob/7eead885b914efcb44e97acfb26a1b81bec6ab64/R/susie_regression.R#L111) for finalizing the ELBO. The univariate and multivariate cases are distinguished by `if ... else` statement.

In [10]:
m_init = mmbr:::MashInitializer$new(list(matrix(0.2*var(dat$y))), 1, alpha=0)
m_init$mash_prior

1.138784


In [12]:
res = mmbr::msusie(dat$X,dat$y,L=L,prior_variance=m_init,compute_objective=T, estimate_residual_variance=F,estimate_prior_variance=F)

In [13]:
res$elbo

[1] -429.4050 -409.5493 -407.7077 -407.7058 -407.7058

The output is also the same as previous calculations.

## ELBO for multivariate calculation

Now using the same code, but more phenotypes, `R = 5`, and still using a very simple prior for the MASH part,

In [14]:
set.seed(2)
dat = mmbr_sim1(r=5)
m_init = mmbr:::MashInitializer$new(list(dat$V), 1, alpha=0)
m_init$mash_prior

1.14068826,-0.02561830,-0.19275481,-0.040421229,-0.029376381
-0.02561830,1.01088663,0.03491952,0.018064842,0.016833489
-0.19275481,0.03491952,1.15215509,-0.112673234,-0.063194855
-0.04042123,0.01806484,-0.11267323,0.836044978,0.008248383
-0.02937638,0.01683349,-0.06319485,0.008248383,0.865937437


In [16]:
res = mmbr::msusie(dat$X,dat$y,L=L,prior_variance=m_init,compute_objective=T,estimate_residual_variance=F,estimate_prior_variance=F)

In [17]:
res$elbo

[1] -2184.076 -2173.199 -2169.551 -2167.744 -2167.267 -2167.087 -2166.514
 [8] -2166.013 -2164.894 -2163.635 -2163.574 -2163.573 -2163.573

and with a different seed:

In [19]:
set.seed(1)
dat = mmbr_sim1(r=5)
m_init = mmbr:::MashInitializer$new(list(dat$V), 1, alpha=0)
res = mmbr::msusie(dat$X,dat$y,L=L,prior_variance=m_init,compute_objective=T,estimate_residual_variance=F,estimate_prior_variance=F)
res$elbo

[1] -2169.123 -2160.895 -2156.179 -2155.080 -2155.045 -2155.043 -2155.043